In [1]:
!pip install wikipedia

In [2]:
from langchain.graph_transformers.diffbot import DiffbotNLPGraphTransformer
from langchain.schema import Document

In [3]:
diffbot_nlp = DiffbotNLPGraphTransformer(diffbot_api_key="")

In [4]:
from langchain.document_loaders import WikipediaLoader

# No chunking because
# Maximum of 100,000 characters per document and 1,000,000 total characters per API request.

raw_documents = WikipediaLoader(query="Leonhard Euler").load()

In [5]:
%timeit
graph_documents = diffbot_nlp.transform_documents(raw_documents)

In [6]:
from langchain.graphs import Neo4jGraph

url="bolt://localhost:7687"
username="neo4j"
password="pleaseletmein"

graph = Neo4jGraph(
    url=url,
    username=username, 
    password=password
)

In [7]:
graph.add_graph_documents(graph_documents)

In [8]:
graph.refresh_schema()

In [12]:
import os
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI

os.environ['OPENAI_API_KEY'] = "sk-"

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True,
)
     

In [14]:
chain.run("Where did Leonhard Euler reside?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Leonhard Euler'})-[:residence]->(l:Location)
RETURN l.name
Full Context:
[{'l.name': 'Saint Petersburg'}, {'l.name': 'Berlin'}]

> Finished chain.


'Leonhard Euler resided in Saint Petersburg and Berlin.'